In [1]:
import sys
sys.path.append('../../')

# BERT_FINE_TUNE_PATH = '../../../bert_fine_tune/'
# sys.path.append(BERT_FINE_TUNE_PATH)

In [2]:
# from textpair.preprocess.dummy_preprocessor import DummyPreprocessor
# from textpair.analyze.bert_analyzer import BertAnalyzer
# from textpair.vectorize.bert_vectorizer import BertVectorizer
# from textpair.model.bert_model import BertModel
# from textpair.pair_ann import PairAnn
# from textpair.semantic.base_semantic import BaseSemantic

import logging

logging.basicConfig(level=logging.INFO)

import torch
from pytorch_pretrained_bert.modeling_fine_tune import BertForPairWiseClassification2
from pytorch_pretrained_bert import BertTokenizer

INFO:pytorch_pretrained_bert.modeling:Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
INFO:pytorch_pretrained_bert.modeling_fine_tune:Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [3]:
FINE_TUNED_PATH = '/efs/fine_tune/lcqmc/pairwise2/lcqmc_fine_tune_40_1_2_1e-5/'

In [4]:
model = BertForPairWiseClassification2.from_pretrained(FINE_TUNED_PATH)
tokenizer = BertTokenizer.from_pretrained(FINE_TUNED_PATH)

INFO:pytorch_pretrained_bert.modeling_fine_tune:loading weights file /efs/fine_tune/lcqmc/pairwise2/lcqmc_fine_tune_40_1_2_1e-5/pytorch_model.bin
INFO:pytorch_pretrained_bert.modeling_fine_tune:loading configuration file /efs/fine_tune/lcqmc/pairwise2/lcqmc_fine_tune_40_1_2_1e-5/config.json
INFO:pytorch_pretrained_bert.modeling_fine_tune:Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "type_vocab_size": 2,
  "vocab_size": 21128
}

INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file /efs/fine_tune/lcqmc/pairwise2/lcqmc_fine_tune_40

In [5]:
def bert_sim(text1, text2):
    tokens1 = ['[CLS]'] + tokenizer.tokenize(text1) + ['[SEP]']
    tokens2 = ['[CLS]'] + tokenizer.tokenize(text2) + ['[SEP]']
    ids1 = tokenizer.convert_tokens_to_ids(tokens1)
    ids2 = tokenizer.convert_tokens_to_ids(tokens2)
    segs1 = [0] * len(ids1)
    segs2 = [0] * len(ids2)
    tokens_tensor1 = torch.tensor([ids1])
    segments_tensor1 = torch.tensor([segs1])
    tokens_tensor2 = torch.tensor([ids2])
    segments_tensor2 = torch.tensor([segs2])
    model.eval()
    with torch.no_grad():
        pos_prob, vec1, vec2 = model(tokens_tensor1, tokens_tensor2, segments_tensor1, segments_tensor2)
    return pos_prob[0].item(), vec1, vec2

# 一般测试

In [6]:
text1 = "我很高兴"
text2 = "我很开心"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.7863309383392334

In [7]:
text1 = "我很开心"
text2 = "我很高兴"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.7863309383392334

In [8]:
text1 = "我很高兴"
text2 = "我特别特别开心"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.7507026195526123

In [9]:
text1 = "我很高兴"
text2 = "我其实觉得自己很开心"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.7257215976715088

In [10]:
text1 = "我特别特别开心"
text2 = "我其实觉得自己很开心"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.8687112331390381

In [11]:
text1 = "我很高兴"
text2 = "我不开心"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.1703854203224182

In [12]:
text1 = "我很高兴"
text2 = "我不高兴"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.14305561780929565

In [13]:
text1 = "我很高兴"
text2 = "我很高兴"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.9999899864196777

In [14]:
text1 = "我很伤心"
text2 = "我很难过"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.8195245265960693

In [15]:
text1 = "真好"
text2 = "不错"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.8848522305488586

In [16]:
text1 = "高兴"
text2 = "开心"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.6417858600616455

In [17]:
text1 = "大家好才是真的好"
text2 = "大家好才是真的好"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.9999899864196777

In [18]:
text1 = "为什么能开出腾讯信用却没有微粒贷朋友的没用腾讯信用却有30000的额度呢"
text2 = "我钱包里没有你们这个应用"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.34980201721191406

In [19]:
text1 = "我也不知道"
text2 = "好吧"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.7652494311332703

In [20]:
text1 = "深度学习"
text2 = "机器学习"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.7974656820297241

In [21]:
text1 = "机器学习"
text2 = "深度学习"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.7974656820297241

In [22]:
text1 = "人民团体是什么"
text2 = "人民团体是指"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.8569705486297607

# 百度-车牌，不太一致

In [23]:
text1 = "车头如何放置车牌"
text2 = "前牌照怎么装"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.23586267232894897

In [24]:
text1 = "车头如何放置车牌"
text2 = "如何办理北京车牌"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.7360151410102844

In [25]:
text1 = "车头如何放置车牌"
text2 = "后牌照怎么装"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.16959434747695923

# 百度-信号

In [26]:
text1 = "信号忽强忽弱"
text2 = "信号忽高忽低"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.6807170510292053

In [27]:
text1 = "信号忽强忽弱"
text2 = "信号忽左忽右"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.2661111354827881

In [28]:
text1 = "信号忽强忽弱"
text2 = "信号忽然中断"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.2763005495071411

# 百度-机器学习

In [29]:
text1 = "如何学好深度学习"
text2 = "深入学习习近平讲话材料"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.12521374225616455

In [30]:
text1 = "如何学好深度学习"
text2 = "机器学习教程"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.56743323802948

In [31]:
text1 = "如何学好深度学习"
text2 = "人工智能教程"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.5748821496963501

# 百度-香蕉的翻译，偏小但排序一致

In [32]:
text1 = "香蕉的翻译"
text2 = "香蕉用英文怎么说"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.8859734535217285

In [33]:
text1 = "香蕉的翻译"
text2 = "香蕉怎么吃"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.55314040184021

In [34]:
text1 = "香蕉的翻译"
text2 = "桔子用英文怎么说"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.8067392110824585

# 百度-腹泻，排序有差别

In [35]:
text1 = "小儿腹泻偏方"
text2 = "宝宝拉肚子偏方"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.6293191909790039

In [36]:
text1 = "小儿腹泻偏方"
text2 = "小儿感冒偏方"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.7290629148483276

In [37]:
text1 = "小儿腹泻偏方"
text2 = "腹泻偏方"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.9012841582298279

# 百度-LOL，数值偏小，但排序一致

In [38]:
text1 = "英雄联盟好玩吗，怎么升级"
text2 = "lol攻略"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.436218798160553

In [39]:
text1 = "英雄联盟好玩吗，怎么升级"
text2 = "英雄联盟服务器升级"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.7534286975860596

In [40]:
text1 = "英雄联盟好玩吗，怎么升级"
text2 = "怎么打好lol"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.6849374175071716

# 百度-红米

In [41]:
text1 = "红米更新出错"
text2 = "红米升级系统出错"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.7065454721450806

In [42]:
text1 = "红米更新出错"
text2 = "红米账户出错"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.7331488728523254

In [43]:
text1 = "红米更新出错"
text2 = "如何买到小米手机"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.24620139598846436

# 百度-李彦宏

In [44]:
text1 = "李彦宏是百度公司创始人"
text2 = "百度是李彦宏创办的"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.8575373888015747

In [45]:
text1 = "李彦宏是百度公司创始人"
text2 = "马化腾创办了腾讯公司"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.8374688625335693

In [46]:
text1 = "李彦宏是百度公司创始人"
text2 = "姚明是NBA的著名球星"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.31878626346588135

# 百度-中国历史

In [47]:
text1 = "中国有五千年的历史"
text2 = "中国是个历史悠久的国家"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.6847238540649414

In [48]:
text1 = "中国有五千年的历史"
text2 = "中国有很多少数民族"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.5159817934036255

In [49]:
text1 = "中国有五千年的历史"
text2 = "中国有13亿人口"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.4089885354042053

# 百度-北京奥运会，偏小，但数值一致

In [50]:
text1 = "北京成功申办了2008年奥运会"
text2 = "2008年奥运会在北京举行"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.5720093846321106

In [51]:
text1 = "北京成功申办了2008年奥运会"
text2 = "伦敦奥运会在2012年举行"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.431368350982666

In [52]:
text1 = "北京成功申办了2008年奥运会"
text2 = "东京奥运会即将举办"
pos_prob, vec1, vec2 = bert_sim(text1, text2)
pos_prob

0.3807816505432129